In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, r2_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier

In [2]:
# load csv
file_load = "../Resources/nhes_19_pfi_cleaned.csv"
learning_data_df = pd.read_csv(file_load)
learning_data_df

,Unnamed: 0,ALLGRADEX_2,ALLGRADEX_3,ALLGRADEX_4,ALLGRADEX_5,ALLGRADEX_6,ALLGRADEX_7,ALLGRADEX_8,ALLGRADEX_9,ALLGRADEX_10,...,CHLDNT_1,CHLDNT_2,CHLDNT_3,CHLDNT_4,LRNCOMP_1,LRNCOMP_2,LRNTAB_1,LRNTAB_2,LRNCELL_1,LRNCELL_2
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,1,1,0
1,4,0,0,0,0,0,0,0,1,0,...,0,1,0,0,1,0,0,1,0,1
2,7,0,0,0,0,0,0,1,0,0,...,1,0,0,0,1,0,1,0,1,0
3,9,0,0,0,0,0,0,0,1,0,...,1,0,0,0,1,0,1,0,0,1
4,11,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8348,16438,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,1,0,1,0
8349,16439,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,1,0,1,0
8350,16441,0,0,0,0,0,1,0,0,0,...,0,1,0,0,1,0,1,0,1,0
8351,16443,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,1,1,0


In [3]:
# drop unnamed and index columns
learning_data_df = learning_data_df.drop(['Unnamed: 0'], axis=1)
learning_data_df

,ALLGRADEX_2,ALLGRADEX_3,ALLGRADEX_4,ALLGRADEX_5,ALLGRADEX_6,ALLGRADEX_7,ALLGRADEX_8,ALLGRADEX_9,ALLGRADEX_10,ALLGRADEX_11,...,CHLDNT_1,CHLDNT_2,CHLDNT_3,CHLDNT_4,LRNCOMP_1,LRNCOMP_2,LRNTAB_1,LRNTAB_2,LRNCELL_1,LRNCELL_2
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,1,1,0
1,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,1,0,0,1,0,1
2,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,1,0,1,0,1,0
3,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,1,0,1,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8348,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,1,0,1,0
8349,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,1,0,1,0
8350,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,1,0,1,0
8351,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,1,1,0


In [4]:
list(learning_data_df.columns)

['ALLGRADEX_2',
 'ALLGRADEX_3',
 'ALLGRADEX_4',
 'ALLGRADEX_5',
 'ALLGRADEX_6',
 'ALLGRADEX_7',
 'ALLGRADEX_8',
 'ALLGRADEX_9',
 'ALLGRADEX_10',
 'ALLGRADEX_11',
 'ALLGRADEX_12',
 'ALLGRADEX_13',
 'ALLGRADEX_14',
 'ALLGRADEX_15',
 'SEGRADES_0',
 'SEGRADES_1',
 'CENREG_1',
 'CENREG_2',
 'CENREG_3',
 'CENREG_4',
 'EDCPUB_1',
 'EDCPUB_2',
 'EDCCAT_1',
 'EDCCAT_2',
 'EDCREL_1',
 'EDCREL_2',
 'EDCPRI_1',
 'EDCPRI_2',
 'EDCINTK12_1',
 'EDCINTK12_2',
 'EDCHSFL_1',
 'EDCHSFL_2',
 'DISTASSI_1',
 'DISTASSI_2',
 'SCHRTSCHL_1',
 'SCHRTSCHL_2',
 'SCHLMAGNET_1',
 'SCHLMAGNET_2',
 'SEENJOY_1',
 'SEENJOY_2',
 'SEENJOY_3',
 'SEENJOY_4',
 'SEABSNT_1',
 'SEABSNT_2',
 'SEABSNT_3',
 'SEABSNT_4',
 'FCSCHOOL_1',
 'FCSCHOOL_2',
 'FCSCHOOL_3',
 'FCSCHOOL_4',
 'FCTEACHR_1',
 'FCTEACHR_2',
 'FCTEACHR_3',
 'FCTEACHR_4',
 'FCSTDS_1',
 'FCSTDS_2',
 'FCSTDS_3',
 'FCSTDS_4',
 'FHHOME_1',
 'FHHOME_2',
 'FHHOME_3',
 'FHHOME_4',
 'FHWKHRS_1',
 'FHWKHRS_2',
 'FHWKHRS_3',
 'FHWKHRS_4',
 'FOSTORY2X_1',
 'FOSTORY2X_2',
 'FO

In [5]:
#Creating a correlation matrix
learning_data_df.corr()

,ALLGRADEX_2,ALLGRADEX_3,ALLGRADEX_4,ALLGRADEX_5,ALLGRADEX_6,ALLGRADEX_7,ALLGRADEX_8,ALLGRADEX_9,ALLGRADEX_10,ALLGRADEX_11,...,CHLDNT_1,CHLDNT_2,CHLDNT_3,CHLDNT_4,LRNCOMP_1,LRNCOMP_2,LRNTAB_1,LRNTAB_2,LRNCELL_1,LRNCELL_2
ALLGRADEX_2,1.000000,-0.006402,-0.032678,-0.034512,-0.038738,-0.041822,-0.043716,-0.048887,-0.050667,-0.051560,...,-0.068595,0.042823,0.060310,-0.012201,-0.126457,0.126457,0.076277,-0.076277,-0.078834,0.078834
ALLGRADEX_3,-0.006402,1.000000,-0.007956,-0.008403,-0.009432,-0.010183,-0.010644,-0.011903,-0.012336,-0.012554,...,-0.029338,0.011517,0.038295,-0.004582,-0.033743,0.033743,0.015063,-0.015063,-0.015423,0.015423
ALLGRADEX_4,-0.032678,-0.007956,1.000000,-0.042893,-0.048145,-0.051977,-0.054332,-0.060759,-0.062970,-0.064080,...,-0.108477,0.071380,0.068013,0.025200,-0.152580,0.152580,0.092211,-0.092211,-0.098630,0.098630
ALLGRADEX_5,-0.034512,-0.008403,-0.042893,1.000000,-0.050848,-0.054895,-0.057382,-0.064170,-0.066505,-0.067678,...,-0.084530,0.054461,0.051958,0.026646,-0.095328,0.095328,0.103295,-0.103295,-0.119507,0.119507
ALLGRADEX_6,-0.038738,-0.009432,-0.048145,-0.050848,1.000000,-0.061617,-0.064408,-0.072027,-0.074649,-0.075965,...,-0.106374,0.080829,0.053405,0.009284,-0.051115,0.051115,0.095523,-0.095523,-0.147114,0.147114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LRNCOMP_2,0.126457,0.033743,0.152580,0.095328,0.051115,0.050869,0.013389,0.001417,-0.029208,-0.044531,...,-0.118696,0.082591,0.057356,0.046304,-1.000000,1.000000,0.122631,-0.122631,-0.111387,0.111387
LRNTAB_1,0.076277,0.015063,0.092211,0.103295,0.095523,0.089603,0.080283,0.018992,-0.020179,-0.029088,...,0.019360,-0.011814,-0.004746,-0.024347,-0.122631,0.122631,1.000000,-1.000000,-0.022079,0.022079
LRNTAB_2,-0.076277,-0.015063,-0.092211,-0.103295,-0.095523,-0.089603,-0.080283,-0.018992,0.020179,0.029088,...,-0.019360,0.011814,0.004746,0.024347,0.122631,-0.122631,-1.000000,1.000000,0.022079,-0.022079
LRNCELL_1,-0.078834,-0.015423,-0.098630,-0.119507,-0.147114,-0.152877,-0.098744,-0.042718,0.009202,0.058511,...,0.210928,-0.166484,-0.093787,-0.019327,0.111387,-0.111387,-0.022079,0.022079,1.000000,-1.000000


In [6]:
# remove allgrade columns and preserve in a separate df
allgrade_df = learning_data_df.filter(['ALLGRADEX_2',
 'ALLGRADEX_3',
 'ALLGRADEX_4',
 'ALLGRADEX_5',
 'ALLGRADEX_6',
 'ALLGRADEX_7',
 'ALLGRADEX_8',
 'ALLGRADEX_9',
 'ALLGRADEX_10',
 'ALLGRADEX_11',
 'ALLGRADEX_12',
 'ALLGRADEX_13',
 'ALLGRADEX_14',
 'ALLGRADEX_15'], axis=1)
allgrade_df.head()

,ALLGRADEX_2,ALLGRADEX_3,ALLGRADEX_4,ALLGRADEX_5,ALLGRADEX_6,ALLGRADEX_7,ALLGRADEX_8,ALLGRADEX_9,ALLGRADEX_10,ALLGRADEX_11,ALLGRADEX_12,ALLGRADEX_13,ALLGRADEX_14,ALLGRADEX_15
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [7]:
# remove allgrade columns from learning_data_df
learning_data_df = learning_data_df.drop([ 'ALLGRADEX_2',
 'ALLGRADEX_3',
 'ALLGRADEX_4',
 'ALLGRADEX_5',
 'ALLGRADEX_6',
 'ALLGRADEX_7',
 'ALLGRADEX_8',
 'ALLGRADEX_9',
 'ALLGRADEX_10',
 'ALLGRADEX_11',
 'ALLGRADEX_12',
 'ALLGRADEX_13',
 'ALLGRADEX_14',
 'ALLGRADEX_15'], axis=1)

In [18]:
learning_data_df

,SEGRADES_0,SEGRADES_1,CENREG_1,CENREG_2,CENREG_3,CENREG_4,EDCPUB_1,EDCPUB_2,EDCCAT_1,EDCCAT_2,...,CHLDNT_1,CHLDNT_2,CHLDNT_3,CHLDNT_4,LRNCOMP_1,LRNCOMP_2,LRNTAB_1,LRNTAB_2,LRNCELL_1,LRNCELL_2
0,0,1,0,0,0,1,1,0,0,1,...,1,0,0,0,1,0,0,1,1,0
1,0,1,0,1,0,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,1
2,0,1,0,1,0,0,1,0,0,1,...,1,0,0,0,1,0,1,0,1,0
3,0,1,0,0,1,0,1,0,0,1,...,1,0,0,0,1,0,1,0,0,1
4,1,0,0,0,1,0,1,0,0,1,...,1,0,0,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8348,0,1,1,0,0,0,1,0,0,1,...,1,0,0,0,1,0,1,0,1,0
8349,0,1,1,0,0,0,1,0,0,1,...,1,0,0,0,1,0,1,0,1,0
8350,0,1,0,0,1,0,1,0,0,1,...,0,1,0,0,1,0,1,0,1,0
8351,0,1,1,0,0,0,1,0,0,1,...,1,0,0,0,1,0,0,1,1,0


In [19]:
learning_data_df.corr()

,SEGRADES_0,SEGRADES_1,CENREG_1,CENREG_2,CENREG_3,CENREG_4,EDCPUB_1,EDCPUB_2,EDCCAT_1,EDCCAT_2,...,CHLDNT_1,CHLDNT_2,CHLDNT_3,CHLDNT_4,LRNCOMP_1,LRNCOMP_2,LRNTAB_1,LRNTAB_2,LRNCELL_1,LRNCELL_2
SEGRADES_0,1.000000,-1.000000,-0.009000,-0.021637,-0.010062,0.042054,-0.030664,0.030664,0.009844,-0.009844,...,-0.051671,0.037214,0.013182,0.040613,-0.065752,0.065752,-0.057370,0.057370,0.013112,-0.013112
SEGRADES_1,-1.000000,1.000000,0.009000,0.021637,0.010062,-0.042054,0.030664,-0.030664,-0.009844,0.009844,...,0.051671,-0.037214,-0.013182,-0.040613,0.065752,-0.065752,0.057370,-0.057370,-0.013112,0.013112
CENREG_1,-0.009000,0.009000,1.000000,-0.339286,-0.239534,-0.249938,0.016178,-0.016178,0.021165,-0.021165,...,0.025541,-0.003055,-0.044289,-0.000729,0.001683,-0.001683,-0.000837,0.000837,-0.008712,0.008712
CENREG_2,-0.021637,0.021637,-0.339286,1.000000,-0.412801,-0.430730,0.008698,-0.008698,-0.019483,0.019483,...,-0.012260,0.004035,0.016542,0.000096,0.000013,-0.000013,0.018300,-0.018300,0.046097,-0.046097
CENREG_3,-0.010062,0.010062,-0.239534,-0.412801,1.000000,-0.304093,0.003365,-0.003365,-0.006559,0.006559,...,0.002060,-0.010678,0.009184,0.014884,-0.028361,0.028361,0.001066,-0.001066,-0.035179,0.035179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LRNCOMP_2,0.065752,-0.065752,-0.001683,-0.000013,0.028361,-0.026251,-0.000070,0.000070,-0.005462,0.005462,...,-0.118696,0.082591,0.057356,0.046304,-1.000000,1.000000,0.122631,-0.122631,-0.111387,0.111387
LRNTAB_1,-0.057370,0.057370,-0.000837,0.018300,0.001066,-0.020966,0.020141,-0.020141,0.002837,-0.002837,...,0.019360,-0.011814,-0.004746,-0.024347,-0.122631,0.122631,1.000000,-1.000000,-0.022079,0.022079
LRNTAB_2,0.057370,-0.057370,0.000837,-0.018300,-0.001066,0.020966,-0.020141,0.020141,-0.002837,0.002837,...,-0.019360,0.011814,0.004746,0.024347,0.122631,-0.122631,-1.000000,1.000000,0.022079,-0.022079
LRNCELL_1,0.013112,-0.013112,-0.008712,0.046097,-0.035179,-0.010073,-0.004362,0.004362,0.006479,-0.006479,...,0.210928,-0.166484,-0.093787,-0.019327,0.111387,-0.111387,-0.022079,0.022079,1.000000,-1.000000


In [8]:
# create features
X = learning_data_df.drop(['SEGRADES_0', 'SEGRADES_1'], axis=1)

# create target
y = learning_data_df['SEGRADES_1']
target_names = ["fail", "pass"]

In [9]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):
    
    #calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    
    return(vif)

In [10]:
#X = learning_data_df.iloc[:,:-1]
vif_df = calc_vif(X)

C:\Users\Jenny\anaconda3\envs\mlenv\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [11]:
vif_df.head(20)

,variables,VIF
0,CENREG_1,2.182329e+08
1,CENREG_2,inf
2,CENREG_3,1.300334e+08
3,CENREG_4,8.479829e+07
4,EDCPUB_1,5.056684e+03
5,EDCPUB_2,1.601180e+10
6,EDCCAT_1,2.849820e+10
7,EDCCAT_2,3.756043e+03
8,EDCREL_1,1.158289e+11
9,EDCREL_2,4.486141e+02


In [12]:
# drop columns with high vif
#learning_data_df = learning_data_df.drop([ 'SCHRTSCHL_1',
 #'SCHRTSCHL_2',
 #'FCSTDS_1',
 #'FCSTDS_2',
 #'FCSTDS_3',
 #'FCSTDS_4',
 #'FOSTORY2X_1',
 #'FOSTORY2X_2',], axis=1)

In [13]:
# prepare data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
# Fit a Random Forest Classifier model, and then print a classification report
clf = RandomForestClassifier(random_state=42).fit(X_train_scaled, y_train)
y_pred = clf.predict(X_test_scaled)
print(classification_report(y_test, y_pred, target_names=target_names))
print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')
print(r2_score(y_test, y_pred))

              precision    recall  f1-score   support

        fail       0.64      0.46      0.54       828
        pass       0.70      0.83      0.76      1261

    accuracy                           0.69      2089
   macro avg       0.67      0.65      0.65      2089
weighted avg       0.68      0.69      0.67      2089

Training Score: 1.0
Testing Score: 0.685016754427956
-0.31649407915656225


In [15]:
# Extremely Random Tree Classifier
clf = ExtraTreesClassifier(random_state=42).fit(X_train_scaled, y_train)
y_pred = clf.predict(X_test_scaled)
print(classification_report(y_test, y_pred, target_names=target_names))
print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')

              precision    recall  f1-score   support

        fail       0.63      0.48      0.54       828
        pass       0.70      0.81      0.75      1261

    accuracy                           0.68      2089
   macro avg       0.66      0.65      0.65      2089
weighted avg       0.67      0.68      0.67      2089

Training Score: 1.0
Testing Score: 0.6802297750119675


In [16]:
# Adaptive Boosting Classifier
clf = AdaBoostClassifier(random_state=42).fit(X_train_scaled, y_train)
print(classification_report(y_test, y_pred, target_names=target_names))
print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')

              precision    recall  f1-score   support

        fail       0.63      0.48      0.54       828
        pass       0.70      0.81      0.75      1261

    accuracy                           0.68      2089
   macro avg       0.66      0.65      0.65      2089
weighted avg       0.67      0.68      0.67      2089

Training Score: 0.70242656449553
Testing Score: 0.697941598851125


In [17]:
def model_tester(model, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = model.fit(X_train_scaled, y_train)
    print(classification_report(y_test, y_pred, target_names=target_names))
    print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
    print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')
    
model_tester(AdaBoostClassifier(random_state=42, n_estimators=100), X, y)
model_tester(AdaBoostClassifier(random_state=42, n_estimators=200), X, y)
model_tester(AdaBoostClassifier(random_state=42, n_estimators=200, learning_rate=0.1), X, y)
model_tester(AdaBoostClassifier(random_state=42, n_estimators=500, learning_rate=0.1), X, y)
model_tester(AdaBoostClassifier(random_state=42, n_estimators=1000, learning_rate=0.1), X, y)
model_tester(AdaBoostClassifier(random_state=42, n_estimators=2000, learning_rate=0.1), X, y)

              precision    recall  f1-score   support

        fail       0.40      0.31      0.35       830
        pass       0.60      0.70      0.65      1259

    accuracy                           0.54      2089
   macro avg       0.50      0.50      0.50      2089
weighted avg       0.52      0.54      0.53      2089

Training Score: 0.7088122605363985
Testing Score: 0.6854954523695548
              precision    recall  f1-score   support

        fail       0.40      0.31      0.35       830
        pass       0.60      0.70      0.65      1259

    accuracy                           0.54      2089
   macro avg       0.50      0.50      0.50      2089
weighted avg       0.52      0.54      0.53      2089

Training Score: 0.7053001277139208
Testing Score: 0.687888942077549
              precision    recall  f1-score   support

        fail       0.40      0.31      0.35       830
        pass       0.60      0.70      0.65      1259

    accuracy                           0.54  